In [16]:
#! pip install nbformat>=4.2.0

In [19]:
import nbformat

In [2]:
#! pip install dash

In [4]:
#! pip install plotly

In [6]:
#pip install pandas

In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import numpy as np

from dash.dependencies import Input, Output
from plotly import graph_objs as go
from plotly.graph_objs import *
from datetime import datetime as dt

import plotly.express as px

import numpy as np

C:\Users\Digby\AppData\Local\Temp\ipykernel_16184\2121921370.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\Digby\AppData\Local\Temp\ipykernel_16184\2121921370.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:
def date_format(x):
    return dt.strptime(x, "%m/%d/%Y").date()

def time_format(x):
    return dt.strptime(x, "%H:%M:%S").time()

In [3]:
df = pd.read_csv('uber-raw-data-apr14.csv')
df[['Date','Time']] = df['Date/Time'].str.split(' ', expand=True)
df['Date'] = df['Date'].apply(date_format)
df['Time'] = df['Time'].apply(time_format)

In [4]:
df = df.sample(n=1000)

In [6]:
# fig = px.scatter_mapbox(df, lat=df['Lat'], lon=df['Lon'], mapbox_style="carto-darkmatter" )
# fig.update_layout(
#     margin=dict(l=0,r=0,b=0,t=0),
#     paper_bgcolor="Black"
#     )
# fig.show()

In [11]:
max_range = max(max(df['Lat']) - min(df['Lat']), max(df['Lon'])-min(df['Lon'])) * 111
zoom = 13.5 - np.log(max_range)
print(zoom)

9.135216171761128


In [23]:
def date_format(x):
    return dt.strptime(x, "%m/%d/%Y").date()

def time_format(x):
    return dt.strptime(x, "%H:%M:%S").time()

def create_map(df):
    max_range = max(max(df['Lat']) - min(df['Lat']), max(df['Lon'])-min(df['Lon'])) * 111
    zoom = 13.5 - np.log(max_range)
    fig = px.scatter_mapbox(df, lat=df['Lat'], lon=df['Lon'], mapbox_style="carto-darkmatter", zoom=zoom )
    fig.update_layout(
        margin=dict(l=0,r=0,b=0,t=10),
        paper_bgcolor="#1E1E1E"
        )
    return fig

def create_chart():
    fig = px.bar(df['Date'].value_counts())
    fig.update_layout(
        margin=dict(l=0,r=0,b=0,t=10),
        paper_bgcolor="#1E1E1E",
        plot_bgcolor="#323130",
        showlegend=False,
        font=dict(color="white"),
        )
    return fig

app = dash.Dash(
    __name__, meta_tags=[{"name": "viewport", "content": "width=device-width"}],
)
app.title = "** GEO RENDERER"
server = app.server


# Plotly mapbox public token
#mapbox_access_token = "pk.eyJ1IjoicGxvdGx5bWFwYm94IiwiYSI6ImNrOWJqb2F4djBnMjEzbG50amg0dnJieG4ifQ.Zme1-Uzoi75IaFbieBDl3A"

# Dictionary of important locations in New York
list_of_locations = {
    "Madison Square Garden": {"lat": 40.7505, "lon": -73.9934},
    "Yankee Stadium": {"lat": 40.8296, "lon": -73.9262},
    "Empire State Building": {"lat": 40.7484, "lon": -73.9857},
    "New York Stock Exchange": {"lat": 40.7069, "lon": -74.0113},
    "JFK Airport": {"lat": 40.644987, "lon": -73.785607},
    "Grand Central Station": {"lat": 40.7527, "lon": -73.9772},
    "Times Square": {"lat": 40.7589, "lon": -73.9851},
    "Columbia University": {"lat": 40.8075, "lon": -73.9626},
    "United Nations HQ": {"lat": 40.7489, "lon": -73.9680},
}

# Initialize data frame
# df = pd.read_csv('uber-raw-data-apr14.csv')
# df[['Date','Time']] = df['Date/Time'].str.split(' ', expand=True)
# df['Date'] = df['Date'].apply(date_format)
# df['Time'] = df['Time'].apply(time_format)

# Layout of Dash App
app.layout = html.Div(
    children=[
        html.Div(
            className="row",
            children=[
                # Column for user controls
                html.Div(
                    className="four columns div-user-controls",
                    children=[
                        html.A(
                            html.Img(
                                className="logo",
                                src=app.get_asset_url("dash-logo-new.png"),
                            ),
                            
                        ),
                        html.H2("GEO RENDERER"),
                        html.P(
                            """Select the time frame for rendering - select an individual point for details."""
                        ),
                        html.Div(
                            className="row",
                            children=[
                                html.Div(
                                    className="div-for-dropdown",
                                    children=[
                                        # Dropdown for locations on map
                                        dcc.Dropdown(
                                            id="time-selector",
                                            options=[
                                                {"label": i, "value": i}
                                                for i in sorted(list(set(df['Date'].unique())))
                                            ],
                                            placeholder="Select a Date", # this will be an option to limit by time (options 24 hr, 12 hr...10mins etc)
                                        )
                                    ],
                                ),
                                html.Div(
                                    className="div-for-dropdown",
                                    children=[
                                        # Dropdown to select times
                                        dcc.Dropdown(
                                            id="bar-selector",
                                            options=[
                                                {
                                                    "label": str(n), 
                                                    "value": str(n),
                                                }
                                                for n in ['Air','Land','Sea'] #these will be options taken from the new DF - dynamically
                                            ],
                                            multi=True,
                                            placeholder="Select specific type",
                                        )
                                    ],
                                ),
                            ],
                        ),
                        html.P(id="total-rides"),
                        html.P(id="total-rides-selection"),

                        # dcc.Slider(0, len(df['Date'].unique()), 1,
                        #            value=len(df['Date'].unique()),
                        #            id='my-slider'
                        # ),
                        html.Div(id='slider-output-container'),


                        
                        #html.P(id="date-value", children="test"),
                        dcc.Markdown(
                            """
                            Clicking a GEO Point will display the inherent data here:
                            """
                        ),
                        html.Br(),
                        html.Div(id='my-output'),
                        html.Div(id="test-output")
                    ],
                ),
                # Column for app graphs and plots
                html.Div(
                    className="eight columns div-for-charts bg-grey",
                    children=[
                        dcc.Graph(id="map-graph", figure=create_map(df)),
                        # html.Div(
                        #     className="text-padding",
                        #     children=[
                        #         "Select any of the bars on the histogram to section data by time."
                        #     ],
                        # ),
                        dcc.Graph(id="histogram", figure = create_chart()),
                    ],
                ),
            ],
        )
    ]
)
# output details of points when clicked
@app.callback(
    Output(component_id='my-output', component_property='children'),
    Input(component_id='map-graph', component_property='clickData'),
    prevent_initial_call = True
)
def update_output_div(clickData):
    return f"{clickData['points'][0]['lat']},{clickData['points'][0]['lon']}"
        #return f"{clickData}"

# update map based on time selection
@app.callback(
    Output(component_id='map-graph', component_property='figure'),
    [
        Input(component_id = 'time-selector', component_property='value'),
        Input(component_id = 'histogram', component_property = 'clickData')
    ],
    prevent_initial_call=True,
)
def filter_map(time_picked, selected_data):
    if time_picked:
        temp_df = df[df['Date']==dt.strptime(time_picked, '%Y-%m-%d').date()]
        max_range = max(max(temp_df['Lat']) - min(temp_df['Lat']), max(temp_df['Lon'])-min(temp_df['Lon'])) * 111
        zoom = 13.5 - np.log(max_range)
        #print(temp_df)
        fig = px.scatter_mapbox(temp_df, lat=temp_df['Lat'], lon=temp_df['Lon'], mapbox_style="carto-darkmatter", zoom=zoom )
        fig.update_layout(
            margin=dict(l=0,r=0,b=0,t=10),
            paper_bgcolor="#1E1E1E"
            )
        return fig
    if selected_data:
        #print(selected_data['points'][0]['label'])
        temp_df = df[df['Date']==dt.strptime(selected_data['points'][0]['label'], '%Y-%m-%d').date()]
        max_range = max(max(temp_df['Lat']) - min(temp_df['Lat']), max(temp_df['Lon'])-min(temp_df['Lon'])) * 111
        zoom = 13.5 - np.log(max_range)
        #print(temp_df)
        fig = px.scatter_mapbox(temp_df, lat=temp_df['Lat'], lon=temp_df['Lon'], mapbox_style="carto-darkmatter", zoom=zoom )
        fig.update_layout(
            margin=dict(l=0,r=0,b=0,t=10),
            paper_bgcolor="#1E1E1E"
            )
        return fig


if __name__ == "__main__":
    app.run_server(jupyter_mode="external",debug=True)

Dash app running on http://127.0.0.1:8050/


In [ ]:
# Gets the amount of days in the specified month
# Index represents month (0 is April, 1 is May, ... etc.)
daysInMonth = [30, 31, 30, 31, 31, 30]

# Get index for the specified month in the dataframe
#monthIndex = pd.Index(["Apr", "May", "June", "July", "Aug", "Sept"])

# Get the amount of rides per hour based on the time selected
# This also higlights the color of the histogram bars based on
# if the hours are selected
def get_selection(month, day, selection):
    xVal = []
    yVal = []
    xSelected = []
    colorVal = [
        "#F4EC15",
        "#DAF017",
        "#BBEC19",
        "#9DE81B",
        "#80E41D",
        "#66E01F",
        "#4CDC20",
        "#34D822",
        "#24D249",
        "#25D042",
        "#26CC58",
        "#28C86D",
        "#29C481",
        "#2AC093",
        "#2BBCA4",
        "#2BB5B8",
        "#2C99B4",
        "#2D7EB0",
        "#2D65AC",
        "#2E4EA4",
        "#2E38A4",
        "#3B2FA0",
        "#4E2F9C",
        "#603099",
    ]

    # Put selected times into a list of numbers xSelected
    xSelected.extend([int(x) for x in selection])

    for i in range(24):
        # If bar is selected then color it white
        if i in xSelected and len(xSelected) < 24:
            colorVal[i] = "#FFFFFF"
        xVal.append(i)
        # Get the number of rides at a particular time
        yVal.append(len(totalList[month][day][totalList[month][day].index.hour == i]))
    return [np.array(xVal), np.array(yVal), np.array(colorVal)]


# Selected Data in the Histogram updates the Values in the Hours selection dropdown menu
@app.callback(
    Output("bar-selector", "value"),
    [Input("histogram", "selectedData"), Input("histogram", "clickData")],
)
def update_bar_selector(value, clickData):
    holder = []
    if clickData:
        holder.append(str(int(clickData["points"][0]["x"])))
    if value:
        for x in value["points"]:
            holder.append(str(int(x["x"])))
    return list(set(holder))


# Clear Selected Data if Click Data is used
@app.callback(Output("histogram", "selectedData"), [Input("histogram", "clickData")])
def update_selected_data(clickData):
    if clickData:
        return {"points": []}


# Update the total number of rides Tag
@app.callback(Output("total-rides", "children"), [Input("date-picker", "date")])
def update_total_rides(datePicked):
    date_picked = dt.strptime(datePicked, "%Y-%m-%d")
    return "Total Number of rides: {:,d}".format(
        len(totalList[date_picked.month - 4][date_picked.day - 1])
    )


# Update the total number of rides in selected times
@app.callback(
    [Output("total-rides-selection", "children"), Output("date-value", "children")],
    [Input("date-picker", "date"), Input("bar-selector", "value")],
)
def update_total_rides_selection(datePicked, selection):
    firstOutput = ""

    if selection is not None or len(selection) is not 0:
        date_picked = dt.strptime(datePicked, "%Y-%m-%d")
        totalInSelection = 0
        for x in selection:
            totalInSelection += len(
                totalList[date_picked.month - 4][date_picked.day - 1][
                    totalList[date_picked.month - 4][date_picked.day - 1].index.hour
                    == int(x)
                ]
            )
        firstOutput = "Total rides in selection: {:,d}".format(totalInSelection)

    if (
        datePicked is None
        or selection is None
        or len(selection) is 24
        or len(selection) is 0
    ):
        return firstOutput, (datePicked, " - showing hour(s): All")

    holder = sorted([int(x) for x in selection])

    if holder == list(range(min(holder), max(holder) + 1)):
        return (
            firstOutput,
            (
                datePicked,
                " - showing hour(s): ",
                holder[0],
                "-",
                holder[len(holder) - 1],
            ),
        )

    holder_to_string = ", ".join(str(x) for x in holder)
    return firstOutput, (datePicked, " - showing hour(s): ", holder_to_string)


# Update Histogram Figure based on Month, Day and Times Chosen
@app.callback(
    Output("histogram", "figure"),
    [Input("date-picker", "date"), Input("bar-selector", "value")],
)
def update_histogram(datePicked, selection):
    date_picked = dt.strptime(datePicked, "%Y-%m-%d")
    monthPicked = date_picked.month - 4
    dayPicked = date_picked.day - 1

    [xVal, yVal, colorVal] = get_selection(monthPicked, dayPicked, selection)

    layout = go.Layout(
        bargap=0.01,
        bargroupgap=0,
        barmode="group",
        margin=go.layout.Margin(l=10, r=0, t=0, b=50),
        showlegend=False,
        plot_bgcolor="#323130",
        paper_bgcolor="#323130",
        dragmode="select",
        font=dict(color="white"),
        xaxis=dict(
            range=[-0.5, 23.5],
            showgrid=False,
            nticks=25,
            fixedrange=True,
            ticksuffix=":00",
        ),
        yaxis=dict(
            range=[0, max(yVal) + max(yVal) / 4],
            showticklabels=False,
            showgrid=False,
            fixedrange=True,
            rangemode="nonnegative",
            zeroline=False,
        ),
        annotations=[
            dict(
                x=xi,
                y=yi,
                text=str(yi),
                xanchor="center",
                yanchor="bottom",
                showarrow=False,
                font=dict(color="white"),
            )
            for xi, yi in zip(xVal, yVal)
        ],
    )

    return go.Figure(
        data=[
            go.Bar(x=xVal, y=yVal, marker=dict(color=colorVal), hoverinfo="x"),
            go.Scatter(
                opacity=0,
                x=xVal,
                y=yVal / 2,
                hoverinfo="none",
                mode="markers",
                marker=dict(color="rgb(66, 134, 244, 0)", symbol="square", size=40),
                visible=True,
            ),
        ],
        layout=layout,
    )


# Get the Coordinates of the chosen months, dates and times
def getLatLonColor(selectedData, month, day):
    listCoords = totalList[month][day]

    # No times selected, output all times for chosen month and date
    if selectedData is None or len(selectedData) is 0:
        return listCoords
    listStr = "listCoords["
    for time in selectedData:
        if selectedData.index(time) is not len(selectedData) - 1:
            listStr += "(totalList[month][day].index.hour==" + str(int(time)) + ") | "
        else:
            listStr += "(totalList[month][day].index.hour==" + str(int(time)) + ")]"
    return eval(listStr)


# Update Map Graph based on date-picker, selected data on histogram and location dropdown
@app.callback(
    Output("map-graph", "figure"),
    [
        Input("date-picker", "date"),
        Input("bar-selector", "value"),
        Input("location-dropdown", "value"),
    ],
)
def update_graph(datePicked, selectedData, selectedLocation):
    zoom = 12.0
    latInitial = 40.7272
    lonInitial = -73.991251
    bearing = 0

    if selectedLocation:
        zoom = 15.0
        latInitial = list_of_locations[selectedLocation]["lat"]
        lonInitial = list_of_locations[selectedLocation]["lon"]

    date_picked = dt.strptime(datePicked, "%Y-%m-%d")
    monthPicked = date_picked.month - 4
    dayPicked = date_picked.day - 1
    listCoords = getLatLonColor(selectedData, monthPicked, dayPicked)

    return go.Figure(
        data=[
            # Data for all rides based on date and time
            Scattermapbox(
                lat=listCoords["Lat"],
                lon=listCoords["Lon"],
                mode="markers",
                hoverinfo="lat+lon+text",
                text=listCoords.index.hour,
                marker=dict(
                    showscale=True,
                    color=np.append(np.insert(listCoords.index.hour, 0, 0), 23),
                    opacity=0.5,
                    size=5,
                    colorscale=[
                        [0, "#F4EC15"],
                        [0.04167, "#DAF017"],
                        [0.0833, "#BBEC19"],
                        [0.125, "#9DE81B"],
                        [0.1667, "#80E41D"],
                        [0.2083, "#66E01F"],
                        [0.25, "#4CDC20"],
                        [0.292, "#34D822"],
                        [0.333, "#24D249"],
                        [0.375, "#25D042"],
                        [0.4167, "#26CC58"],
                        [0.4583, "#28C86D"],
                        [0.50, "#29C481"],
                        [0.54167, "#2AC093"],
                        [0.5833, "#2BBCA4"],
                        [1.0, "#613099"],
                    ],
                    colorbar=dict(
                        title="Time of<br>Day",
                        x=0.93,
                        xpad=0,
                        nticks=24,
                        tickfont=dict(color="#d8d8d8"),
                        titlefont=dict(color="#d8d8d8"),
                        thicknessmode="pixels",
                    ),
                ),
            ),
            # Plot of important locations on the map
            Scattermapbox(
                lat=[list_of_locations[i]["lat"] for i in list_of_locations],
                lon=[list_of_locations[i]["lon"] for i in list_of_locations],
                mode="markers",
                hoverinfo="text",
                text=[i for i in list_of_locations],
                marker=dict(size=8, color="#ffa0a0"),
            ),
        ],
        layout=Layout(
            autosize=True,
            margin=go.layout.Margin(l=0, r=35, t=0, b=0),
            showlegend=False,
            mapbox=dict(
                accesstoken=mapbox_access_token,
                center=dict(lat=latInitial, lon=lonInitial),  # 40.7272  # -73.991251
                style="dark",
                bearing=bearing,
                zoom=zoom,
            ),
            updatemenus=[
                dict(
                    buttons=(
                        [
                            dict(
                                args=[
                                    {
                                        "mapbox.zoom": 12,
                                        "mapbox.center.lon": "-73.991251",
                                        "mapbox.center.lat": "40.7272",
                                        "mapbox.bearing": 0,
                                        "mapbox.style": "dark",
                                    }
                                ],
                                label="Reset Zoom",
                                method="relayout",
                            )
                        ]
                    ),
                    direction="left",
                    pad={"r": 0, "t": 0, "b": 0, "l": 0},
                    showactive=False,
                    type="buttons",
                    x=0.45,
                    y=0.02,
                    xanchor="left",
                    yanchor="bottom",
                    bgcolor="#323130",
                    borderwidth=1,
                    bordercolor="#6d6d6d",
                    font=dict(color="#FFFFFF"),
                )
            ],
        ),
    )


if __name__ == "__main__":
    app.run_server(debug=True)

<>:259: SyntaxWarning:

"is not" with 'int' literal. Did you mean "!="?

<>:274: SyntaxWarning:

"is" with 'int' literal. Did you mean "=="?

<>:275: SyntaxWarning:

"is" with 'int' literal. Did you mean "=="?

<>:370: SyntaxWarning:

"is" with 'int' literal. Did you mean "=="?

<>:259: SyntaxWarning:

"is not" with 'int' literal. Did you mean "!="?

<>:274: SyntaxWarning:

"is" with 'int' literal. Did you mean "=="?

<>:275: SyntaxWarning:

"is" with 'int' literal. Did you mean "=="?

<>:370: SyntaxWarning:

"is" with 'int' literal. Did you mean "=="?

C:\Users\Digby\AppData\Local\Temp\ipykernel_8648\2704952999.py:259: SyntaxWarning:

"is not" with 'int' literal. Did you mean "!="?

C:\Users\Digby\AppData\Local\Temp\ipykernel_8648\2704952999.py:274: SyntaxWarning:

"is" with 'int' literal. Did you mean "=="?

C:\Users\Digby\AppData\Local\Temp\ipykernel_8648\2704952999.py:275: SyntaxWarning:

"is" with 'int' literal. Did you mean "=="?

C:\Users\Digby\AppData\Local\Temp\ipykernel_8648\

---------------------------------------------------------------------------
NotFound                                  Traceback (most recent call last)
NotFound: 404 Not Found: The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
Cell In[17], line 247, in update_total_rides(datePicked='2014-04-01')
    243 @app.callback(Output("total-rides", "children"), [Input("date-picker", "date")])
    244 def update_total_rides(datePicked):
    245     date_picked = dt.strptime(datePicked, "%Y-%m-%d")
    246     return "Total Number of rides: {:,d}".format(
--> 247         len(totalList[date_picked.month - 4][date_picked.day - 1])
        date_picked = datetime.datetime(2014, 4, 1, 0, 0)
    248     )

NameError: name 'totalList' is not defined

----------------------------------